<a href="https://colab.research.google.com/github/jaadukanna/Chat-with-PDF-Chatbot/blob/main/kafka_pyspark_mongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=4f70f06f4aea124bed61011ea081b1a538d6bfe336d73662576fcc9070dfa58f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Check Spark Session Information
spark

In [ ]:
# Import a Spark function from library
from pyspark.sql.functions import col

In [ ]:
# install kafka-python
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Download latest stable kafka archive
!curl -sSOL https://downloads.apache.org/kafka/3.5.1/kafka_2.13-3.5.1.tgz
!tar -xzf kafka_2.13-3.5.1.tgz

In [ ]:
!./kafka_2.13-3.5.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.1/config/zookeeper.properties
!./kafka_2.13-3.5.1/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

root        3376       1  0 08:59 ?        00:00:11 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.5.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.5.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.5.1/bin/../config/log4j.properties -cp /content/kafka_2.13-3.5.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.5.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.5.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.5.1/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.13-3.5.1/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.5.1/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.1

In [ ]:
# Create Kafka Topic
!./kafka_2.13-3.5.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic hello_topic



Created topic hello_topic.


In [ ]:
# Describe the topic
!./kafka_2.13-3.5.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic hello_topic

Topic: hello_topic	TopicId: mej55sb6Qs22hv5dKOERWA	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: hello_topic	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [ ]:
import pandas as pd
df = pd.read_csv("movies.csv")
df = df.iloc[[11, 14, 16]]
df

,movieId,title,genres
11,12,Dracula: Dead and Loving It (1995),Comedy|Horror
14,15,Cutthroat Island (1995),Action|Adventure|Romance
16,17,Sense and Sensibility (1995),Drama|Romance


In [ ]:
import pandas as pd
from kafka import KafkaProducer
# Initialize Kafka Producer
producer = KafkaProducer(bootstrap_servers='127.0.0.1:9092')
# Convert DataFrame to JSON and send to Kafka topic
for _, row in df.iterrows():
    producer.send('hello_topic', value=row.to_json().encode('utf-8'))

In [ ]:
!./kafka_2.13-3.5.1/bin/kafka-console-producer.sh --bootstrap-server 127.0.0.1:9092 --topic data_topic

In [ ]:
!./kafka_2.13-3.5.1/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic hello_topic --from-beginning

Processed a total of 9 messages


In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.13/3.4.1/spark-sql-kafka-0-10_2.13-3.4.1.jar"

--2023-08-18 12:36:11--  https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.13/3.4.1/spark-sql-kafka-0-10_2.13-3.4.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435459 (425K) [application/java-archive]
Saving to: ‘spark-sql-kafka-0-10_2.13-3.4.1.jar’

spark-sql-kafka-0-1 100%[===================>] 425.25K  --.-KB/s    in 0.04s   

2023-08-18 12:36:11 (9.73 MB/s) - ‘spark-sql-kafka-0-10_2.13-3.4.1.jar’ saved [435459/435459]



In [ ]:
import os
!ls
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

kafka_2.13-3.5.1      movies.csv
kafka_2.13-3.5.1.tgz  sample_data
ml_ratings.csv	      spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType, IntegerType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("StreamingDataProcessing") \
    .getOrCreate()

# Define the schema for the streaming data
schema = StructType().add("movieId", IntegerType()).add("title", StringType()).add("genres", StringType())

# Read streaming data from Kafka topic
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
    .option("subscribe", "hello_topic") \
    .load()

# Convert the value column from binary to string
df = df.withColumn("value", df["value"].cast(StringType()))

# Parse the JSON data
df = df.withColumn("data", from_json(df["value"], schema))

# Select the required columns
df = df.select("data.movieId", "data.title", "data.genres")

# Write the processed data to the console
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the query to terminate
query.awaitTermination()

AnalysisException: ignored

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar -xvf spark-3.4.1-bin-hadoop3.tgz
!pip install findspark

In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [ ]:

!./spark-3.4.1-bin-hadoop3/bin/spark-submit --jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar --master spark://127.0.0.1:7077 --deploy-mode client ./demospark.py


In [ ]:
!./spark-3.4.1-bin-hadoop3/bin/pyspark

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/18 12:59:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/18 12:59:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/

Using Python version 3.10.12 (main, Jun 11 2023 05:26:28)
Spark context Web UI available at http://440107d07693:4041
Spark context available as 'sc' (master = local[*], app id = local-1692363597686).
SparkSession available as 'spark'.
>>> 

In [ ]:
!ls /content/spark-3.4.1-bin-hadoop3/bin

beeline		      pyspark		spark-class.cmd      spark-shell.cmd
beeline.cmd	      pyspark2.cmd	spark-connect-shell  spark-sql
docker-image-tool.sh  pyspark.cmd	sparkR		     spark-sql2.cmd
find-spark-home       run-example	sparkR2.cmd	     spark-sql.cmd
find-spark-home.cmd   run-example.cmd	sparkR.cmd	     spark-submit
load-spark-env.cmd    spark-class	spark-shell	     spark-submit2.cmd
load-spark-env.sh     spark-class2.cmd	spark-shell2.cmd     spark-submit.cmd
